In [8]:
import pandas as pd
import requests, json
import time

from tqdm import tqdm

from server import *

In [3]:
with open("data/matchid.json", "r", encoding="utf-8") as file:
    
    matchid_list = json.load(file)  # JSON 데이터를 파이썬 리스트로 변환

print(matchid_list)

['6778de039378b50150a583b2', '677d11cd832e06a5bb83e4af', '677bb3a65e4ed79f523d521b', '6779432dd08f3a16ef92f49a', '677ce6b1d9303f50e0aa9f5e', '677b67017248fa1b29fa1204', '677cf15d0e66065b94398bc5', '677bebd6dbcf449ac440389f', '677c14d5a5ae7f185642872f', '6779560b75b8f11417f4b27f', '67788c544eb5a2e91f8ddbd6', '677ce26326bdfc8308336cc2', '677c880d854bd68236f2942f', '677cf65550e22a14a6c4d891', '677a75f5630348000250cf95', '677b95be1a42b632d1449d73', '677c8ab1be080d0e1ab2b35e', '677cf02ed704f2a6b99df875', '67741d800d1039efc8ff04ac', '677cde45c08e5d20a6a50414', '6779b5758a8ae076f3344ac6', '67794c20056d392cb5b4f465', '677d0824ed1e48866c1a4237', '677bebd1b27358469dc84ab9', '677c2a73023e002ea77e6708', '677534e2c88d9d4648ff1c5c', '67750b3f33518cb707f6bf67', '677cb41ed74693a74ad74b3e', '6778f0ace052333409888bd7', '677c7276ab59494d82370167', '677a391c9a5b5f85e3b9bb1e', '67756c60217c5773fb59c875', '677d0279f43f532009acfc50', '677c2518e5038760c6e11fda', '677aa1f045d8ea7042639c87', '677ce157f56d700837

In [ ]:
func = MATHCH_FUNCTION['detail']
df = pd.DataFrame()
error_list = []

# 요청 간 대기 시간 (초)
REQUEST_DELAY = 3
# 최대 재시도 횟수
MAX_RETRIES = 3

for matchid in tqdm(matchid_list):
    urlString = basic_link + func + "?matchid=" + matchid

    for attempt in range(MAX_RETRIES):
        try:
            # API 호출
            response = requests.get(urlString, headers=headers)
            result = response.json()

            # API 호출이 성공한 경우
            matchinfo = result['matchInfo']
            break
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                print(f"Error for matchid {matchid}, retrying ({attempt + 1}/{MAX_RETRIES})...")
                time.sleep(REQUEST_DELAY)  # 재시도 전 대기
            else:
                print(f"Failed to retrieve data for matchid {matchid} after {MAX_RETRIES} attempts.")
                error_list.append(matchid)
                matchinfo = None

    # matchinfo가 None이면 다음 루프 진행
    if not matchinfo:
        continue

    # 데이터 처리
    for p in range(len(matchinfo)):
        target = matchinfo[p]['player']

        for i in range(len(target)):
            data = target[i]

            # 'status'를 펼쳐서 단일 레벨 딕셔너리로 변환
            flattened_data = {
                **{'spId': data['spId'], 'spPosition': data['spPosition'], 'spGrade': data['spGrade']},
                **data['status']
            }

            # 데이터프레임으로 변환
            tmp = pd.DataFrame([flattened_data])

            # 기존 데이터프레임과 병합
            df = pd.concat([df, tmp], axis=0)

    # 요청 간 대기 시간
    time.sleep(REQUEST_DELAY)

In [ ]:
df.to_csv('data/matchdetail.csv')